In [1]:
from accounts import si

#si.ID, si.PW

In [ ]:
!pip install BeautifulSoup4
!pip install selenium
!pip install selenium --upgrade
!pip install webdriver-manager

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
import time

url = "https://www.melon.com/index.htm"
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
driver= "msedgedriver.exe"
service=Service(executable_path=driver)
#webdriver 객체 생성
browser = webdriver.Edge(service=service)

#melon 이동
browser.get(url)

#
#리뷰 더보기 버튼 찾기 및 클릭
melon_btn = browser.find_element(By.CSS_SELECTOR,"#gnb_menu > ul:nth-child(1) > li.nth3 > a > span.menu_bg.menu03") 
melon_btn.click()
#
melon_btn2 = browser.find_element(By.CSS_SELECTOR,"#conts > div.wrap_tabmenu01.type08 > ul > li:nth-child(5) > a")
melon_btn2.click()

melon_btn3 = browser.find_element(By.CSS_SELECTOR,"#conts > div.wrap_tabmenu02.type06 > ul > li:nth-child(4) > a")
melon_btn3.click()
# 

#     tag_list = soup.select("#frm div > ul > li > div.entry > div.info > a") # 찾아내는 목표 설정.
#     print(len(tag_list))
title_list=[]
sec_title_list=[]
review_list=[]

i=0
while i==3: #멜론 3페이지 검사 위해
    i+=1
    html = browser.page_source
    soup = BeautifulSoup(html, "lxml")# html 파싱

    #큰 제목 뽑기
    title_tag_list = soup.select("#frm > div > ul > li > div.entry > div.info > a") # 파싱된 html 내용에서   
    for title in title_tag_list:
        title_list.append(title.get_text().strip)
    
    #부제목 뽑기
    sec_title_tag_list=soup.select('frm > div > ul > li > div.reveiw > h4')
    for sec in sec_title_tag_list:
        sec_title_list.append(sec.get_text().strip)
    #리뷰 뽑기
    review_tag_list= soup.select('#frm > div > ul > li > div.reveiw > p')
    for review in review_tag_list:#     tag_list = soup.select("#frm div > ul > li > div.entry > div.info > a") # 찾아내는 목표 설정.
        review_list.append(review.get_text().strip)
    print(len(title_list))
    print(len(sec_title_list))
    print(len(review_list))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#conts > div.wrap_tabmenu01.type08 > ul > li:nth-child(5) > a"}
  (Session info: MicrosoftEdge=114.0.1823.67); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF664B6AA52+64226]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF664AFBC12+765378]
	(No symbol) [0x00007FF6648BCC4C]
	(No symbol) [0x00007FF664900BDF]
	(No symbol) [0x00007FF664900D6A]
	(No symbol) [0x00007FF66493AC17]
	(No symbol) [0x00007FF66491F03F]
	(No symbol) [0x00007FF6648F4BB1]
	(No symbol) [0x00007FF664937FC1]
	(No symbol) [0x00007FF66491EDD3]
	(No symbol) [0x00007FF6648F3BEC]
	(No symbol) [0x00007FF6648F2DD6]
	(No symbol) [0x00007FF6648F4364]
	Microsoft::Applications::Events::ILogManager::DispatchEventBroadcast [0x00007FF664D48359+131903

In [3]:
import os
from datetime import datetime
import time
import requests
import re

import pandas as pd

from bs4 import BeautifulSoup

import selenium


from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
'''
class SRICrawler():
사람인 -> url ->pc status code 200
0.셀레늄이용 먼저 로그인 엔터 누르기 
1.검색창에 입력값 (회사이름) 입력 시 
->회사 리뷰까지 접속 
->코멘트를 뭐였지 아무튼 검사로 체크해서 들어가기
->
'''

In [4]:
service = Service(executable_path='chromedriver.exe')
browser = webdriver.Chrome(service=service)

In [5]:
url = r"https://www.saramin.co.kr/zf_user/"

browser.maximize_window()
print(browser.get_window_size())

browser.get(url)

{'width': 1552, 'height': 832}


In [6]:

#로그인 버튼 클릭
login_screen_btn = browser.find_element(By.CSS_SELECTOR,"#sri_header > div.wrap_header > div.utility > div.sign > a.btn_sign.signin") 
login_screen_btn.click()
#아이디 버튼 클릭
id_btn = browser.find_element(By.CSS_SELECTOR,"#id")
#id_btn.click()
#아이디 입력
id_btn.send_keys(si.ID)

#비밀번호 버튼 클릭
pwd_btn = browser.find_element(By.CSS_SELECTOR,"#password")
pwd_btn.click()
#비밀번호 입력
pwd_btn.send_keys(si.PW)

#로그인 클릭
login_btn = browser.find_element(By.CSS_SELECTOR,"#login_frm > div > div > div.login-form > button").send_keys(Keys.ENTER)

In [7]:
#메인 홈페이지로 돌아가기
main_btn = browser.find_element(By.CSS_SELECTOR,'#sri_gnb_wrap > a > svg.img_ci')
main_btn.click()


In [20]:

#검색창 클릭
search_btn = browser.find_element(By.CSS_SELECTOR,'#btn_search')
search_btn.click()

#검색창 2 클릭
search_btn2 = browser.find_element(By.CSS_SELECTOR,'#ipt_keyword_recruit')
search_btn2.click()

In [21]:
#검색어 입력
cop=input()
search_btn2.send_keys(cop)


In [22]:
#검색어 엔터
enter_btn = browser.find_element(By.CSS_SELECTOR,'#btn_search_recruit')
enter_btn.click()


In [23]:
#(주) 앞뒤로 포함돼 있고 대상 검색 문자열(cop)과 일치
i=1
while True:
    try: #(주)가 붙은회사
        n_btn = browser.find_element(By.CSS_SELECTOR, f"div:nth-child({i}) > h2 > a") #뒤에 (주)글자까지 포함
        
        
        if cop+'(주)' == n_btn.text or '(주)'+cop == n_btn.text: #주 붙은 회사 먼저 while문 크롤링   
            n_btn.click() #주가 붙어있는 회사라면 클릭   
            break       
        i += 1
                
            
    except: # (주)가 붙지 않은 회사
        i = 1 #주 붙지 않은 회사 찾기 위해 초기화
        while True:
            try: # 
                n_btn2 = browser.find_element(By.CSS_SELECTOR, f"div:nth-child({i}) > h2 > a > b") #(주)나 뒤에붙는 파생단어들 떼짐
                
                
                if cop == n_btn2.text: #주 붙지 않고 검색어와 일치하는 회사   
                    n_btn2.click()  #일치하는 회사 클릭   
                    break       
                i += 1
                        
                    
            except: #안쪽 try 구문에서도 오류가 뜬다는 것은 더 이상 cop 데이터가 없다는것으로 오류 처리
                print(f"No result found for {cop}")
                break
        break        

In [24]:
#검색 엔진 알고리즘은 어느정도 완성했고 방향만 첫 메인화면에서 기업.연봉 카테고리에서 기업/리뷰 클릭으로 검색들어가는걸로 바꿈

In [19]:
#execute_script("자바스크립트 코드")=> 스크롤 내리기
#browser.execute_script("window.scrollTo(0,1000)")